<a href="https://colab.research.google.com/github/janimiyarj/Normal-Tissue-prediction-using-Bert-Model/blob/main/Normal_Tissue_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [2]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00


In [58]:
df = pd.read_csv('/content/normal_tissue.tsv', sep='\t')
df = df.head(10000)

In [5]:
df.head()

,Gene,Gene name,Tissue,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,Approved
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved


In [6]:
df.shape

(10000, 6)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Gene         10000 non-null  object
 1   Gene name    10000 non-null  object
 2   Tissue       9998 non-null   object
 3   Cell type    9998 non-null   object
 4   Level        9998 non-null   object
 5   Reliability  10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


In [8]:
df.isnull().sum()

,0
Gene,0
Gene name,0
Tissue,2
Cell type,2
Level,2
Reliability,0


In [9]:
# Fill missing values with the mode of each column
for column in df.columns:
    # Calculate the mode for the column
    mode_value = df[column].mode()
    # Fill NaN values with the mode (taking the first mode if multiple)
    if not mode_value.empty:
        df[column].fillna(mode_value[0], inplace=True)

<ipython-input-9-451d89c354c3>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value[0], inplace=True)


In [10]:
df.isnull().sum()

,0
Gene,0
Gene name,0
Tissue,0
Cell type,0
Level,0
Reliability,0


In [60]:
df['Tissue'].value_counts().head(5)

,count
Tissue,
skin 1,545
testis,486
cerebellum,461
cerebral cortex,452
colon,363


In [12]:
df['Gene'].value_counts()

,count
Gene,
ENSG00000002586,159
ENSG00000003436,135
ENSG00000006118,134
ENSG00000002834,129
ENSG00000005893,128
...,...
ENSG00000003402,76
ENSG00000004779,73
ENSG00000006282,60


In [13]:
df['Level'].value_counts()

,count
Level,
Not detected,4611
Medium,2610
Low,1732
High,1046
Ascending,1


I have assumed my target as Level

In [14]:
df['Gene name'].value_counts()

,count
Gene name,
CD99,159
TFPI,135
TMEM132A,134
LASP1,129
LAMP2,128
...,...
CFLAR,76
NDUFAB1,73
SPATA20,60


In [15]:
df.head()

,Gene,Gene name,Tissue,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,Approved
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved


In [61]:
df['Cell type'].value_counts().head(5)

,count
Cell type,
glandular cells,2155
squamous epithelial cells,547
neuronal cells,335
glial cells,335
fibroblasts,331


In [17]:
df['Reliability'].value_counts()

,count
Reliability,
Approved,3625
Enhanced,3052
Supported,1910
Uncertain,1413


In [18]:
df['Reliability'].replace({'Approved': 0, 'Enhanced': 1, 'Uncertain': 2, 'Supported': 3}, inplace=True)

<ipython-input-18-be5be02d7c9e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Reliability'].replace({'Approved': 0, 'Enhanced': 1, 'Uncertain': 2, 'Supported': 3}, inplace=True)
<ipython-input-18-be5be02d7c9e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Reliability'].replace({'Approv

In [19]:
df['Reliability'].tail()

,Reliability
9995,1
9996,1
9997,1
9998,1
9999,1


Reliability will be my Output feature

In [20]:
X = df.drop('Reliability', axis=1)
y = df['Reliability']

In [21]:
print(X.shape)
print(y.shape)

(10000, 5)
(10000,)


In [22]:
print(X.head())
print('\n')
print(y.head())

              Gene Gene name          Tissue            Cell type  \
0  ENSG00000000003    TSPAN6  adipose tissue           adipocytes   
1  ENSG00000000003    TSPAN6   adrenal gland      glandular cells   
2  ENSG00000000003    TSPAN6        appendix      glandular cells   
3  ENSG00000000003    TSPAN6        appendix      lymphoid tissue   
4  ENSG00000000003    TSPAN6     bone marrow  hematopoietic cells   

          Level  
0  Not detected  
1  Not detected  
2        Medium  
3  Not detected  
4  Not detected  


0    0
1    0
2    0
3    0
4    0
Name: Reliability, dtype: int64


In [23]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(8000, 5)
(2000, 5)
(8000,)
(2000,)


In [25]:

# Load BERT for sequence classification
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
df.head()

,Gene,Gene name,Tissue,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,0
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,0
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,0
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,0
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,0


In [27]:

# Combine text columns for BERT input
X_train['text'] = X_train['Tissue'] + ' ' + X_train['Cell type'] + ' ' + X_train['Gene'] + ' ' + X_train['Level']
X_test['text'] = X_test['Tissue'] + ' ' + X_test['Cell type'] + ' ' + X_test['Gene'] + ' ' + X_test['Level']


In [28]:
print(X_train['text'].head())
print('\n')
print(X_test['text'].head())

9254     stomach 1 glandular cells ENSG00000006062 Medium
1561      appendix lymphoid tissue ENSG00000002549 Medium
1670    appendix endocrine cells ENSG00000002586 Not d...
6087    oral mucosa squamous epithelial cells ENSG0000...
6669    fallopian tube ciliated cells (tip of cilia) E...
Name: text, dtype: object


6252       kidney cells in glomeruli ENSG00000005175 High
4684    pancreas pancreatic endocrine cells ENSG000000...
1731       epididymis glandular cells ENSG00000002586 Low
4742    endometrium 1 glandular cells ENSG00000004864 Low
4521         kidney cells in tubules ENSG00000004779 High
Name: text, dtype: object


In [29]:
# Tokenize the inputs
train_encodings = tokenizer(X_train['text'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test['text'].tolist(), truncation=True, padding=True, max_length=128)

In [37]:
#train_encodings

In [31]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
eval_use_gather_object=False,
evaluation_strategy=epoch,
fp

In [38]:
# Create datasets using the Hugging Face `datasets` library
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'],
                                    'attention_mask': train_encodings['attention_mask'],
                                    'labels': Y_train})

test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'],
                                   'attention_mask': test_encodings['attention_mask'],
                                   'labels': Y_test})


In [46]:

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [42]:
trainer.train()

# Evaluate the model
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,0.655800,0.005798
2,0.007500,0.000852
3,0.002500,0.000619


{'eval_loss': 0.000619018916040659,
 'eval_runtime': 5.6258,
 'eval_samples_per_second': 355.507,
 'eval_steps_per_second': 22.219,
 'epoch': 3.0}

In [43]:
# Make predictions
predictions = trainer.predict(test_dataset)
y_predicted = np.argmax(predictions.predictions, axis=1)


In [44]:
print("Predicted classes:", y_predicted)

Predicted classes: [1 2 3 ... 2 0 2]


In [45]:
# Print the first example in the dataset
print("First example:")
print(test_dataset[0])

# Print the first two examples
print("\nFirst two examples:")
print(test_dataset[:2])

# Convert to a Pandas DataFrame and print it
import pandas as pd

df = test_dataset.to_pandas()
print("\nDataset as a DataFrame:")
print(df)

# Select and print specific rows
selected_rows = test_dataset.select([0, 1, 2])
print("\nSelected rows:")
print(selected_rows)

First example:
{'input_ids': [101, 14234, 4442, 1999, 1043, 21297, 2121, 15859, 4372, 28745, 8889, 8889, 8889, 2692, 22203, 23352, 2152, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 1}

First two examples:
{'input_ids': [[101, 14234, 4442, 1999, 1043, 21297, 2121, 15859, 4372, 28745, 8889, 8889, 8889, 2692, 22203, 23352, 2152, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 6090, 16748, 3022, 6090, 16748, 12070, 2203, 10085, 11467, 4442, 4372, 28745, 8889, 8889, 8889, 2692, 18139, 2692, 2683, 2025, 11156, 102, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]], 'labels': [1, 2]}

Dataset as a DataFrame:
                                              input_ids  \
0     [101, 14234, 4442, 1999, 1043, 21297, 2121, 15... 

In [49]:
# Define your input
input_text = ['kidney cells in glomeruli ENSG00000005175 High']

# Tokenize the input
inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized input to a dataset
dataset = Dataset.from_dict(inputs)

# Make predictions
predictions = trainer.predict(dataset)

In [50]:
predictions

PredictionOutput(predictions=array([[-2.5146272,  6.7058415, -2.2173016, -2.195495 ]], dtype=float32), label_ids=None, metrics={'test_model_preparation_time': 0.0061, 'test_runtime': 0.0706, 'test_samples_per_second': 14.171, 'test_steps_per_second': 14.171})

In [51]:
y_predicted_value = np.argmax(predictions.predictions, axis=1)

In [53]:
y_predicted_value


array([1])

In [55]:
reverse_mapping = {0: 'Approved', 1: 'Enhanced', 2: 'Uncertain', 3: 'Supported'}


# Convert the predicted value to text
predicted_label = reverse_mapping[y_predicted_value[0]]  # Access the first element if it's an array

# Display the predicted label
print(predicted_label)

Enhanced
